import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, lit
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 15:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/23 15:02:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


read csv with inferschema

In [ ]:
data = spark.read.csv("/Users/BigTime/Documents/spark_demo/ds_salaries.csv", header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [ ]:
df = spark.read.csv("/Users/BigTime/Documents/spark_demo/ds_salaries.csv", header=True, inferSchema=True)

write schema of scv on screen

In [ ]:
data.printSchema()

create schema of this scv

In [3]:
data_schema = [
               StructField('_c0', IntegerType(), True),
               StructField('work_year', IntegerType(), True),
               StructField('experience_level', StringType(), True),
               StructField('employment_type', StringType(), True),
               StructField('job_title', StringType(), True),
               StructField('salary', IntegerType(), True),
               StructField('salary_currency', StringType(), True),
               StructField('salary_in_usd', IntegerType(), True),
               StructField('employee_residence', StringType(), True),
               StructField('remote_ratio', IntegerType(), True),
               StructField('company_location', StringType(), True),
               StructField('company_size', StringType(), True),
            ]

final_struc = StructType(fields = data_schema)

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [4]:
data = spark.read.csv("/Users/BigTime/Documents/spark_demo/ds_salaries.csv", header=True, schema=final_struc)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [5]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [6]:
data.show

<bound method DataFrame.show of DataFrame[_c0: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]>

print data in dataframe using display(df.toPandas())

In [7]:
data.toPandas()

24/07/23 15:03:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [8]:
df_job_title = data.dropDuplicates(["job_title"])

print all rows from df_job_titles without truncating jobs

In [9]:
df_job_title.show(truncate=False)

24/07/23 15:03:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|job_title                         |salary |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|77 |2021     |MI              |PT             |3D Computer Vision Researcher     |400000 |INR            |5409         |IN                |50          |IN              |M           |
|52 |2020     |EN              |FT             |AI Scientist                      |300000 |DKK            |45896        |DK                |50          |DK              |S           |
|344|2022     |EX              |FT             |Analytics Engineer              

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [10]:
df_analytic = data.groupBy("job_title").agg(
    avg("salary_in_usd").alias("avg_salary"),
    min("salary_in_usd").alias("min_salary"),
    max("salary_in_usd").alias("max_salary"))

print all rows from df_analytic without trancating jobs

In [11]:
df_analytic.show(truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [12]:
w = Window.orderBy(df_analytic.avg_salary.desc())
df_analytic = df_analytic.withColumn("row_id", row_number().over(w))

print all data from df_analytic

In [13]:
df_analytic.show(truncate=False)

24/07/23 15:03:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------------------+------------------+----------+----------+------+
|job_title                         |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------+------------------+----------+----------+------+
|Data Analytics Lead               |405000.0          |405000    |405000    |1     |
|Principal Data Engineer           |328333.3333333333 |185000    |600000    |2     |
|Financial Data Analyst            |275000.0          |100000    |450000    |3     |
|Principal Data Scientist          |215242.42857142858|148261    |416000    |4     |
|Director of Data Science          |195074.0          |130026    |325000    |5     |
|Data Architect                    |177873.9090909091 |90700     |266400    |6     |
|Applied Data Scientist            |175655.0          |54238     |380000    |7     |
|Analytics Engineer                |175000.0          |135000    |205300    |8     |
|Data Specialist                   |165000.0          |165000    

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [14]:
df_analytic = df_analytic.select('row_id', 'job_title', 'avg_salary', 'min_salary', 'max_salary')


print df_analytic now

In [15]:
df_analytic.show(truncate=False)

24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+----------------------------------+------------------+----------+----------+
|row_id|job_title                         |avg_salary        |min_salary|max_salary|
+------+----------------------------------+------------------+----------+----------+
|1     |Data Analytics Lead               |405000.0          |405000    |405000    |
|2     |Principal Data Engineer           |328333.3333333333 |185000    |600000    |
|3     |Financial Data Analyst            |275000.0          |100000    |450000    |
|4     |Principal Data Scientist          |215242.42857142858|148261    |416000    |
|5     |Director of Data Science          |195074.0          |130026    |325000    |
|6     |Data Architect                    |177873.9090909091 |90700     |266400    |
|7     |Applied Data Scientist            |175655.0          |54238     |380000    |
|8     |Analytics Engineer                |175000.0          |135000    |205300    |
|9     |Data Specialist                   |165000.0          |165

24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/23 15:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [16]:
max_salary_df = data.groupBy("experience_level").agg(max("salary_in_usd").alias("biggest_salary"))
max_salary_df = max_salary_df.withColumnRenamed('experience_level', 'to_drop')

df_exp_lvl = data.join(max_salary_df, (data.experience_level == max_salary_df.to_drop), 
                     "inner").drop('to_drop')

print here df_exp_lvl

In [17]:
df_exp_lvl.toPandas()

24/07/23 15:03:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,biggest_salary
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L,450000
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S,412000
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M,412000
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S,450000
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L,412000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M,412000
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M,412000
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M,412000
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M,412000


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [18]:
df_best = df_exp_lvl.filter(df_exp_lvl.salary_in_usd == df_exp_lvl.biggest_salary).select('_c0','experience_level', 'biggest_salary', 'employee_residence')
df_best = df_best.withColumnRenamed('_c0', 'id')

print df_best

In [19]:
df_best.toPandas()

24/07/23 15:03:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


,id,experience_level,biggest_salary,employee_residence
0,33,MI,450000,US
1,37,EN,250000,US
2,63,SE,412000,US
3,97,MI,450000,US
4,252,EX,600000,US


drop duplicates if exist by experience_level

In [20]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [21]:
df_best.toPandas()

24/07/23 15:03:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


,id,experience_level,biggest_salary,employee_residence
0,37,EN,250000,US
1,252,EX,600000,US
2,33,MI,450000,US
3,63,SE,412000,US


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [22]:
df_new_best = df_best.drop('id')
df_new_best = df_new_best.withColumn("experience_level", \
   when((df_new_best.experience_level == 'MI'), lit("middle")) \
     .when((df_new_best.experience_level == 'SE'), lit("senior")) \
     .otherwise(lit("Null")) \
  )

print df_new_best

In [23]:
df_new_best.toPandas()

,experience_level,biggest_salary,employee_residence
0,Null,250000,US
1,Null,600000,US
2,middle,450000,US
3,senior,412000,US


write df_new_best like 1.csv and load then it to df_final

In [26]:
df_new_best.write.csv("/Users/BigTime/Documents/spark_demo/ds_salaries_res.csv")

fin_schema = [
               StructField('experience_level', StringType(), True),
               StructField('biggest_salary', IntegerType(), True),
               StructField('employee_residence', StringType(), True),
            ]

fin_struc = StructType(fields = fin_schema)

df_final = spark.read.csv("/Users/BigTime/Documents/spark_demo/ds_salaries_res.csv", schema=fin_struc)

print df_final

In [27]:
df_final.toPandas()

,experience_level,biggest_salary,employee_residence
0,Null,250000,US
1,Null,600000,US
2,middle,450000,US
3,senior,412000,US


In [28]:
# filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

df_final = df_final.filter(df_final.experience_level != 'Null')
df_final = data.join(df_final, (df_final.biggest_salary == data.salary_in_usd) & (df_final.employee_residence == data.employee_residence))

print df_final

In [29]:
df_final.toPandas()

24/07/23 15:04:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///Users/BigTime/Documents/spark_demo/ds_salaries.csv


,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,experience_level,biggest_salary,employee_residence
0,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M,middle,450000,US
1,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L,senior,412000,US
2,97,2021,MI,FT,Financial Data Analyst,450000,USD,450000,US,100,US,L,middle,450000,US


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [30]:
biggest_salary_in_usd = df_final.agg({"biggest_salary":"max"}).first()[0]
print(biggest_salary_in_usd)

450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system